In [4]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/Users/atintaya/MyProjects/data_analysis_bikes/kaggle.json"

In [ ]:
import pandas as pd
import zipfile

In [5]:
# Downloading dataset from kaggle
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

Traceback (most recent call last):
  File "/Users/atintaya/MyProjects/data_analysis_bikes/env/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Users/atintaya/MyProjects/data_analysis_bikes/env/lib/python3.9/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/Users/atintaya/MyProjects/data_analysis_bikes/env/lib/python3.9/site-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /Users/atintaya/MyProjects/data_analysis_bikes/kaggle.json. Or use the environment method.
